In [1]:
# path = '/home/dpanina/share/rete_tools/examples/'
# path = '/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/'

In [2]:
import sys
# # sys.path.insert(0, '/home/dpanina/retentioneering-tools-new-arch/')
#
sys.path.insert(0, '..')

# Инициация графа

In [3]:
import pandas as pd

from src.eventstream.eventstream import Eventstream
from src.eventstream.schema import RawDataSchema, EventstreamSchema
from src.graph.p_graph import PGraph, EventsNode

raw_data = pd.read_csv( 'simple-onlineshop.csv')

raw_data_schema = RawDataSchema(
                            event_name="event", 
                            event_timestamp="timestamp", 
                            user_id="user_id")

source = Eventstream(
    raw_data=raw_data,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

graph = PGraph(source_stream=source)


In [4]:
raw_data_schema

RawDataSchema(event_name='event', event_timestamp='timestamp', user_id='user_id', event_type=None, custom_cols=[])

In [5]:
raw_data.head(8)

,user_id,event,timestamp
0,219483890,catalog,2019-11-01 17:59:13.273932
1,219483890,product1,2019-11-01 17:59:28.459271
2,219483890,cart,2019-11-01 17:59:29.502214
3,219483890,catalog,2019-11-01 17:59:32.557029
4,964964743,catalog,2019-11-01 21:38:19.283663
5,964964743,cart,2019-11-01 21:38:36.761221
6,964964743,delivery_choice,2019-11-01 21:38:37.564693
7,629881394,main,2019-11-01 22:28:54.791683


In [6]:
source.to_dataframe().head(8)

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,0,catalog,2019-11-01 17:59:13.273932,219483890
1,072c19d0-427b-4278-809b-c509e52cee4a,raw,1,product1,2019-11-01 17:59:28.459271,219483890
2,1692b904-c338-473e-b00b-5b16d5c79d67,raw,2,cart,2019-11-01 17:59:29.502214,219483890
3,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,3,catalog,2019-11-01 17:59:32.557029,219483890
4,78809682-25ea-4d09-b8fb-008c6da653e1,raw,4,catalog,2019-11-01 21:38:19.283663,964964743
5,94c484be-8f3e-4578-b43c-ef62184132ca,raw,5,cart,2019-11-01 21:38:36.761221,964964743
6,9930949d-a839-4358-863d-f3e16d77f02b,raw,6,delivery_choice,2019-11-01 21:38:37.564693,964964743
7,56c73621-a726-43fc-b1e5-a286c93afd85,raw,7,main,2019-11-01 22:28:54.791683,629881394


In [7]:
source_df = source.to_dataframe()

In [8]:
source_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35381 entries, 0 to 35380
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   event_id         35381 non-null  object        
 1   event_type       35381 non-null  object        
 2   event_index      35381 non-null  int64         
 3   event_name       35381 non-null  object        
 4   event_timestamp  35381 non-null  datetime64[ns]
 5   user_id          35381 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.9+ MB


## Результат - тут все общие вопросы

user_id должен быть int
можем ли задавать эту схему типов?
Почему каждая функция - это отдельный класс?
Почему это не может быть класс препроцессинга?
- get_parents() - выдача не оч информативная
    - [<src.graph.p_graph.EventsNode at 0x7f3892298160>]
- как можно посмотреть все предыдущие ноды до корня?

 # Добавление синтетических событий

## StartEnd

In [9]:
from src.data_processors_lib.rete import StartEndEvents, StartEndEventsParams

In [10]:
start_end_events = EventsNode(
    StartEndEvents(params=StartEndEventsParams(**{}))
)

graph.add_node(
    node=start_end_events,
    parents=[graph.root]
)

result = graph.combine(
    node=start_end_events
)
result.to_dataframe().head(8)

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,0246ba24-8115-47e0-8d7c-4b3339987e6a,start,0,start,2019-11-01 17:59:13.273932,219483890.0
1,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,1,catalog,2019-11-01 17:59:13.273932,219483890.0
2,072c19d0-427b-4278-809b-c509e52cee4a,raw,2,product1,2019-11-01 17:59:28.459271,219483890.0
3,1692b904-c338-473e-b00b-5b16d5c79d67,raw,3,cart,2019-11-01 17:59:29.502214,219483890.0
4,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,4,catalog,2019-11-01 17:59:32.557029,219483890.0
5,f2506613-b1d8-4eed-a98e-f99f6bc019e9,start,5,start,2019-11-01 21:38:19.283663,964964743.0
6,78809682-25ea-4d09-b8fb-008c6da653e1,raw,6,catalog,2019-11-01 21:38:19.283663,964964743.0
7,94c484be-8f3e-4578-b43c-ef62184132ca,raw,7,cart,2019-11-01 21:38:36.761221,964964743.0


In [11]:
df = result.to_dataframe()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42883 entries, 0 to 42882
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   event_id         42883 non-null  object        
 1   event_type       42883 non-null  object        
 2   event_index      42883 non-null  int64         
 3   event_name       42883 non-null  object        
 4   event_timestamp  42883 non-null  datetime64[ns]
 5   user_id          42883 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 2.3+ MB


In [12]:
print(len(df[df['user_id'] == 219483890]))
df[df['user_id'] == 219483890]

19


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,0246ba24-8115-47e0-8d7c-4b3339987e6a,start,0,start,2019-11-01 17:59:13.273932,219483890.0
1,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,1,catalog,2019-11-01 17:59:13.273932,219483890.0
2,072c19d0-427b-4278-809b-c509e52cee4a,raw,2,product1,2019-11-01 17:59:28.459271,219483890.0
3,1692b904-c338-473e-b00b-5b16d5c79d67,raw,3,cart,2019-11-01 17:59:29.502214,219483890.0
4,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,4,catalog,2019-11-01 17:59:32.557029,219483890.0
2715,b2f9c0a1-904a-4dff-90e1-37be04986437,raw,2715,main,2019-12-06 16:22:57.484842,219483890.0
2716,1436f264-7a18-46b6-a732-60201d8b56d8,raw,2716,catalog,2019-12-06 16:23:01.331109,219483890.0
2717,7019613a-00ba-4aa8-b08d-3118638d9c02,raw,2717,catalog,2019-12-06 16:23:48.116617,219483890.0
5721,ebdfbd61-4fcd-44ff-b562-649e3efc76f8,raw,5721,main,2020-01-06 22:10:13.635011,219483890.0
5722,a927160b-96ce-4c89-bd53-d79a37bdd1b6,raw,5722,catalog,2020-01-06 22:10:15.228575,219483890.0


In [13]:
print(len(source_df[source_df['user_id'] == 219483890]))
source_df[source_df['user_id'] == 219483890].sort_values('event_timestamp')

17


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,0,catalog,2019-11-01 17:59:13.273932,219483890
1,072c19d0-427b-4278-809b-c509e52cee4a,raw,1,product1,2019-11-01 17:59:28.459271,219483890
2,1692b904-c338-473e-b00b-5b16d5c79d67,raw,2,cart,2019-11-01 17:59:29.502214,219483890
3,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,3,catalog,2019-11-01 17:59:32.557029,219483890
2245,b2f9c0a1-904a-4dff-90e1-37be04986437,raw,2245,main,2019-12-06 16:22:57.484842,219483890
2246,1436f264-7a18-46b6-a732-60201d8b56d8,raw,2246,catalog,2019-12-06 16:23:01.331109,219483890
2247,7019613a-00ba-4aa8-b08d-3118638d9c02,raw,2247,catalog,2019-12-06 16:23:48.116617,219483890
4836,ebdfbd61-4fcd-44ff-b562-649e3efc76f8,raw,4836,main,2020-01-06 22:10:13.635011,219483890
4837,a927160b-96ce-4c89-bd53-d79a37bdd1b6,raw,4837,catalog,2020-01-06 22:10:15.228575,219483890
4838,2fa7934a-4e76-4018-994e-1dac03a4464f,raw,4838,cart,2020-01-06 22:10:42.309028,219483890


In [14]:
check_id = source_df[source_df['user_id'] == 219483890]['event_id'][0]
df[df['event_id'] == check_id]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
1,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,1,catalog,2019-11-01 17:59:13.273932,219483890.0


In [15]:
source_df[source_df['event_id'] == check_id]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,0,catalog,2019-11-01 17:59:13.273932,219483890


In [16]:
source_df['user_id'].nunique(), df['user_id'].nunique()

(3751, 3751)

In [17]:
df[df['event_type'] == 'start']['event_type'].count(), df[df['event_type'] == 'end']['event_type'].count()

(3751, 3751)

In [18]:
df[df['event_type'] == 'raw']['event_type'].count(), source.to_dataframe()['event_type'].count()

(35381, 35381)

In [19]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [20]:
df['event_type'].value_counts()

raw      35381
start     3751
end       3751
Name: event_type, dtype: int64

In [21]:
len(source_df), len(df)

(35381, 42883)

### Результат

1) [поправила] Исчезло событие, которое до этого было первым с event_id = '556697a2-7439-45c1-bd70-8d970ab7ef61'
Тоже самое произошло с событием End
result.to_dataframe(show_deleted=True) - то все нормально

2) [поправила] Количество raw событий после combine уменьшилось ровно на кол-во пользователей*2
Общая длина eventstream при этом не изменилась
3) [TODO] #params - не должно быть обязательных параметров

## NewResume

In [22]:
from src.data_processors_lib.rete import NewResumeEvents, NewResumeParams

# new_user_list = [219483890]
new_user_list = 'all'
new_resume_events = EventsNode(
    NewResumeEvents(params=NewResumeParams(new_users_list=new_user_list)))


graph.add_node(
    node=new_resume_events,
    parents=[graph.root]
)

result = graph.combine(
    node=new_resume_events
)
result.to_dataframe().head(8)

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,561fd68b-240d-462c-9b4f-6154afc31f5c,new_user,0,new_user,2019-11-01 17:59:13.273932,219483890.0
1,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,1,catalog,2019-11-01 17:59:13.273932,219483890.0
2,072c19d0-427b-4278-809b-c509e52cee4a,raw,2,product1,2019-11-01 17:59:28.459271,219483890.0
3,1692b904-c338-473e-b00b-5b16d5c79d67,raw,3,cart,2019-11-01 17:59:29.502214,219483890.0
4,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,4,catalog,2019-11-01 17:59:32.557029,219483890.0
5,e9181881-4906-4349-b2c5-073141943245,new_user,5,new_user,2019-11-01 21:38:19.283663,964964743.0
6,78809682-25ea-4d09-b8fb-008c6da653e1,raw,6,catalog,2019-11-01 21:38:19.283663,964964743.0
7,94c484be-8f3e-4578-b43c-ef62184132ca,raw,7,cart,2019-11-01 21:38:36.761221,964964743.0


In [23]:
df1 = result.to_dataframe()

In [24]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [25]:
df1['event_type'].value_counts()

raw         35381
new_user     3751
Name: event_type, dtype: int64

In [26]:
len(df1), len(source_df)

(39132, 35381)

In [27]:
len(df1) - len(source_df)

3751

In [28]:
graph.get_parents(new_resume_events)

In [29]:
print(len(source_df[source_df['user_id'] == 219483890]))
source_df[source_df['user_id'] == 219483890].sort_values('event_timestamp')

17


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,0,catalog,2019-11-01 17:59:13.273932,219483890
1,072c19d0-427b-4278-809b-c509e52cee4a,raw,1,product1,2019-11-01 17:59:28.459271,219483890
2,1692b904-c338-473e-b00b-5b16d5c79d67,raw,2,cart,2019-11-01 17:59:29.502214,219483890
3,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,3,catalog,2019-11-01 17:59:32.557029,219483890
2245,b2f9c0a1-904a-4dff-90e1-37be04986437,raw,2245,main,2019-12-06 16:22:57.484842,219483890
2246,1436f264-7a18-46b6-a732-60201d8b56d8,raw,2246,catalog,2019-12-06 16:23:01.331109,219483890
2247,7019613a-00ba-4aa8-b08d-3118638d9c02,raw,2247,catalog,2019-12-06 16:23:48.116617,219483890
4836,ebdfbd61-4fcd-44ff-b562-649e3efc76f8,raw,4836,main,2020-01-06 22:10:13.635011,219483890
4837,a927160b-96ce-4c89-bd53-d79a37bdd1b6,raw,4837,catalog,2020-01-06 22:10:15.228575,219483890
4838,2fa7934a-4e76-4018-994e-1dac03a4464f,raw,4838,cart,2020-01-06 22:10:42.309028,219483890


In [30]:
print(len(df1[df1['user_id'] == 219483890]))
df1[df1['user_id'] == 219483890].sort_values('event_timestamp')

18


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,561fd68b-240d-462c-9b4f-6154afc31f5c,new_user,0,new_user,2019-11-01 17:59:13.273932,219483890.0
1,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,1,catalog,2019-11-01 17:59:13.273932,219483890.0
2,072c19d0-427b-4278-809b-c509e52cee4a,raw,2,product1,2019-11-01 17:59:28.459271,219483890.0
3,1692b904-c338-473e-b00b-5b16d5c79d67,raw,3,cart,2019-11-01 17:59:29.502214,219483890.0
4,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,4,catalog,2019-11-01 17:59:32.557029,219483890.0
2553,b2f9c0a1-904a-4dff-90e1-37be04986437,raw,2553,main,2019-12-06 16:22:57.484842,219483890.0
2554,1436f264-7a18-46b6-a732-60201d8b56d8,raw,2554,catalog,2019-12-06 16:23:01.331109,219483890.0
2555,7019613a-00ba-4aa8-b08d-3118638d9c02,raw,2555,catalog,2019-12-06 16:23:48.116617,219483890.0
5388,ebdfbd61-4fcd-44ff-b562-649e3efc76f8,raw,5388,main,2020-01-06 22:10:13.635011,219483890.0
5389,a927160b-96ce-4c89-bd53-d79a37bdd1b6,raw,5389,catalog,2020-01-06 22:10:15.228575,219483890.0


In [31]:
check_id = source_df[source_df['user_id'] == 219483890]['event_id'][0]
df1[df1['event_id'] == check_id]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
1,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,1,catalog,2019-11-01 17:59:13.273932,219483890.0


In [32]:
print(len(source_df[source_df['user_id'] == 964964743]))
source_df[source_df['user_id'] == 964964743].sort_values('event_timestamp')

13


,event_id,event_type,event_index,event_name,event_timestamp,user_id
4,78809682-25ea-4d09-b8fb-008c6da653e1,raw,4,catalog,2019-11-01 21:38:19.283663,964964743
5,94c484be-8f3e-4578-b43c-ef62184132ca,raw,5,cart,2019-11-01 21:38:36.761221,964964743
6,9930949d-a839-4358-863d-f3e16d77f02b,raw,6,delivery_choice,2019-11-01 21:38:37.564693,964964743
2439,eff948e1-e0e8-4afb-b52f-08936264a027,raw,2439,main,2019-12-09 01:42:22.801831,964964743
2440,46834714-9baf-45b4-9be2-0604ae8a93bb,raw,2440,catalog,2019-12-09 01:42:23.617764,964964743
2441,323eaaa9-f74c-4b27-8f53-215ac900ac43,raw,2441,product2,2019-12-09 01:42:56.877340,964964743
2442,a7137007-a5d6-43af-9461-d28828bee74e,raw,2442,catalog,2019-12-09 01:43:05.436223,964964743
2443,44d254c7-3f8f-45a0-9348-42865bc01961,raw,2443,catalog,2019-12-09 01:43:36.923178,964964743
2444,dc6ba294-693e-499d-810f-8b37e8704856,raw,2444,product2,2019-12-09 01:43:41.174195,964964743
2445,9405e65d-7c43-4473-b1ae-1f38550976f2,raw,2445,cart,2019-12-09 01:43:57.325569,964964743


In [33]:
print(len(df1[df1['user_id'] == 964964743]))
df1[df1['user_id'] == 964964743].sort_values('event_timestamp')

14


,event_id,event_type,event_index,event_name,event_timestamp,user_id
5,e9181881-4906-4349-b2c5-073141943245,new_user,5,new_user,2019-11-01 21:38:19.283663,964964743.0
6,78809682-25ea-4d09-b8fb-008c6da653e1,raw,6,catalog,2019-11-01 21:38:19.283663,964964743.0
7,94c484be-8f3e-4578-b43c-ef62184132ca,raw,7,cart,2019-11-01 21:38:36.761221,964964743.0
8,9930949d-a839-4358-863d-f3e16d77f02b,raw,8,delivery_choice,2019-11-01 21:38:37.564693,964964743.0
2768,eff948e1-e0e8-4afb-b52f-08936264a027,raw,2768,main,2019-12-09 01:42:22.801831,964964743.0
2769,46834714-9baf-45b4-9be2-0604ae8a93bb,raw,2769,catalog,2019-12-09 01:42:23.617764,964964743.0
2770,323eaaa9-f74c-4b27-8f53-215ac900ac43,raw,2770,product2,2019-12-09 01:42:56.877340,964964743.0
2771,a7137007-a5d6-43af-9461-d28828bee74e,raw,2771,catalog,2019-12-09 01:43:05.436223,964964743.0
2772,44d254c7-3f8f-45a0-9348-42865bc01961,raw,2772,catalog,2019-12-09 01:43:36.923178,964964743.0
2773,dc6ba294-693e-499d-810f-8b37e8704856,raw,2773,product2,2019-12-09 01:43:41.174195,964964743.0


### зададим ноды последовательно

In [34]:
new_user_list = [219483890]

new_resume_events1 = EventsNode(
    NewResumeEvents(params=NewResumeParams(new_users_list=new_user_list)))


graph.add_node(
    node=new_resume_events1,
    parents=[start_end_events]
)

result_seq = graph.combine(
    node=new_resume_events1
)
result_seq.to_dataframe().head(8)

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,05b47748-3954-44f4-a6fa-0df1e8ce61e4,start,0,start,2019-11-01 17:59:13.273932,219483890.0
1,24502e81-6b73-426c-ac86-70bc5a1e66a9,new_user,1,new_user,2019-11-01 17:59:13.273932,219483890.0
2,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,2,catalog,2019-11-01 17:59:13.273932,219483890.0
3,072c19d0-427b-4278-809b-c509e52cee4a,raw,3,product1,2019-11-01 17:59:28.459271,219483890.0
4,1692b904-c338-473e-b00b-5b16d5c79d67,raw,4,cart,2019-11-01 17:59:29.502214,219483890.0
5,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,5,catalog,2019-11-01 17:59:32.557029,219483890.0
6,eba8ed4a-26b1-4aec-9b07-45a5f847ac62,start,6,start,2019-11-01 21:38:19.283663,964964743.0
7,5d4f5c59-6244-4d15-9e92-7456fa79b257,resume,7,resume,2019-11-01 21:38:19.283663,964964743.0


In [35]:
len(result_seq.to_dataframe()), len(source_df)

(46634, 35381)

In [36]:
result_seq.to_dataframe()['event_type'].value_counts()

raw         35381
start        3751
end          3751
resume       3750
new_user        1
Name: event_type, dtype: int64

In [37]:
result.to_dataframe()['event_type'].value_counts()

raw         35381
new_user     3751
Name: event_type, dtype: int64

In [38]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [39]:
df2 = result_seq.to_dataframe()

In [40]:
print(len(df2[df2['user_id'] == 219483890]))
df2[df2['user_id'] == 219483890].sort_values('event_timestamp')

20


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,05b47748-3954-44f4-a6fa-0df1e8ce61e4,start,0,start,2019-11-01 17:59:13.273932,219483890.0
1,24502e81-6b73-426c-ac86-70bc5a1e66a9,new_user,1,new_user,2019-11-01 17:59:13.273932,219483890.0
2,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,2,catalog,2019-11-01 17:59:13.273932,219483890.0
3,072c19d0-427b-4278-809b-c509e52cee4a,raw,3,product1,2019-11-01 17:59:28.459271,219483890.0
4,1692b904-c338-473e-b00b-5b16d5c79d67,raw,4,cart,2019-11-01 17:59:29.502214,219483890.0
5,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,5,catalog,2019-11-01 17:59:32.557029,219483890.0
3023,b2f9c0a1-904a-4dff-90e1-37be04986437,raw,3023,main,2019-12-06 16:22:57.484842,219483890.0
3024,1436f264-7a18-46b6-a732-60201d8b56d8,raw,3024,catalog,2019-12-06 16:23:01.331109,219483890.0
3025,7019613a-00ba-4aa8-b08d-3118638d9c02,raw,3025,catalog,2019-12-06 16:23:48.116617,219483890.0
6273,ebdfbd61-4fcd-44ff-b562-649e3efc76f8,raw,6273,main,2020-01-06 22:10:13.635011,219483890.0


In [41]:
graph.get_parents(new_resume_events1)

### Результат

1) [Поправила] Обсуждали, что можно задать вот так тип new_users_list: List[int] | str
у меня это не сработало
В итоге поправила - new_users_list: Union[str, List[int]]
2) [TODO] Можно ли как-то проставить конкретное значение "all"?
Пока добавила отбивку
3) [Поправила] combiner
    - Задвоились все raw события, кроме первого,
    - Первое событие удалилось по аналогии со start_end_events, проверила по id
    -new_user и resume проставился правильно c правильным timestamp

- Обсудить с Володей
При задании new_resume_events, должна быть возможность задать параметр "all" - чтобы всем проставить new_user - нужен он такой? нужен ли такой для resume?

## LostPause

In [42]:
from src.data_processors_lib.rete import LostPauseEvents, LostPauseParams

lost_users_list_ = [964964743]

lost_pause_events = EventsNode(
    LostPauseEvents(params=LostPauseParams(lost_users_list=lost_users_list_)))

graph.add_node(
    node=lost_pause_events,
    parents=[graph.root]
)


result = graph.combine(
    node=lost_pause_events
)


result.to_dataframe().head(8)

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,0,catalog,2019-11-01 17:59:13.273932,219483890.0
1,072c19d0-427b-4278-809b-c509e52cee4a,raw,1,product1,2019-11-01 17:59:28.459271,219483890.0
2,1692b904-c338-473e-b00b-5b16d5c79d67,raw,2,cart,2019-11-01 17:59:29.502214,219483890.0
3,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,3,catalog,2019-11-01 17:59:32.557029,219483890.0
4,78809682-25ea-4d09-b8fb-008c6da653e1,raw,4,catalog,2019-11-01 21:38:19.283663,964964743.0
5,94c484be-8f3e-4578-b43c-ef62184132ca,raw,5,cart,2019-11-01 21:38:36.761221,964964743.0
6,9930949d-a839-4358-863d-f3e16d77f02b,raw,6,delivery_choice,2019-11-01 21:38:37.564693,964964743.0
7,56c73621-a726-43fc-b1e5-a286c93afd85,raw,7,main,2019-11-01 22:28:54.791683,629881394.0


In [43]:
df3 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
print(df3.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())

0
0


In [44]:
print(len(df3[df3['user_id'] == 964964743]))
df3[df3['user_id'] == 964964743]

14


,event_id,event_type,event_index,event_name,event_timestamp,user_id
4,78809682-25ea-4d09-b8fb-008c6da653e1,raw,4,catalog,2019-11-01 21:38:19.283663,964964743.0
5,94c484be-8f3e-4578-b43c-ef62184132ca,raw,5,cart,2019-11-01 21:38:36.761221,964964743.0
6,9930949d-a839-4358-863d-f3e16d77f02b,raw,6,delivery_choice,2019-11-01 21:38:37.564693,964964743.0
2618,eff948e1-e0e8-4afb-b52f-08936264a027,raw,2618,main,2019-12-09 01:42:22.801831,964964743.0
2619,46834714-9baf-45b4-9be2-0604ae8a93bb,raw,2619,catalog,2019-12-09 01:42:23.617764,964964743.0
2620,323eaaa9-f74c-4b27-8f53-215ac900ac43,raw,2620,product2,2019-12-09 01:42:56.877340,964964743.0
2621,a7137007-a5d6-43af-9461-d28828bee74e,raw,2621,catalog,2019-12-09 01:43:05.436223,964964743.0
2622,44d254c7-3f8f-45a0-9348-42865bc01961,raw,2622,catalog,2019-12-09 01:43:36.923178,964964743.0
2623,dc6ba294-693e-499d-810f-8b37e8704856,raw,2623,product2,2019-12-09 01:43:41.174195,964964743.0
2624,9405e65d-7c43-4473-b1ae-1f38550976f2,raw,2624,cart,2019-12-09 01:43:57.325569,964964743.0


In [45]:
print(len(source_df[source_df['user_id'] == 964964743]))
source_df[source_df['user_id'] == 964964743]

13


,event_id,event_type,event_index,event_name,event_timestamp,user_id
4,78809682-25ea-4d09-b8fb-008c6da653e1,raw,4,catalog,2019-11-01 21:38:19.283663,964964743
5,94c484be-8f3e-4578-b43c-ef62184132ca,raw,5,cart,2019-11-01 21:38:36.761221,964964743
6,9930949d-a839-4358-863d-f3e16d77f02b,raw,6,delivery_choice,2019-11-01 21:38:37.564693,964964743
2439,eff948e1-e0e8-4afb-b52f-08936264a027,raw,2439,main,2019-12-09 01:42:22.801831,964964743
2440,46834714-9baf-45b4-9be2-0604ae8a93bb,raw,2440,catalog,2019-12-09 01:42:23.617764,964964743
2441,323eaaa9-f74c-4b27-8f53-215ac900ac43,raw,2441,product2,2019-12-09 01:42:56.877340,964964743
2442,a7137007-a5d6-43af-9461-d28828bee74e,raw,2442,catalog,2019-12-09 01:43:05.436223,964964743
2443,44d254c7-3f8f-45a0-9348-42865bc01961,raw,2443,catalog,2019-12-09 01:43:36.923178,964964743
2444,dc6ba294-693e-499d-810f-8b37e8704856,raw,2444,product2,2019-12-09 01:43:41.174195,964964743
2445,9405e65d-7c43-4473-b1ae-1f38550976f2,raw,2445,cart,2019-12-09 01:43:57.325569,964964743


In [46]:
print(len(source_df[source_df['user_id'] == 629881394]))
# source_df[source_df['user_id'] == 629881394]
print(len(df3[df3['user_id'] == 629881394]))
df3[df3['user_id'] == 629881394]

48
49


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,56c73621-a726-43fc-b1e5-a286c93afd85,raw,7,main,2019-11-01 22:28:54.791683,629881394.0
9,a445bdd8-3c0d-4b0b-b894-a692f45fbbe0,raw,9,catalog,2019-11-01 22:29:01.049513,629881394.0
11,1170cc7c-64d4-49cf-abeb-32b78a595f19,raw,11,catalog,2019-11-01 22:29:32.322458,629881394.0
13,c1f3b671-ca82-4770-b9b8-3faae2c2e76a,raw,13,catalog,2019-11-01 22:30:09.450839,629881394.0
14,c54f2fcc-2fca-4323-8a54-779fdcdaa11d,raw,14,catalog,2019-11-01 22:31:05.565762,629881394.0
15,3217c680-2e11-47c1-bf45-dabb92c8ab65,raw,15,main,2019-11-01 22:31:08.333560,629881394.0
16,66df2664-a3a3-48fe-abe9-2a2d4e85a62c,raw,16,catalog,2019-11-01 22:31:09.010626,629881394.0
17,b2689f02-542c-4390-8733-7c701afaac0d,raw,17,product1,2019-11-01 22:31:10.416231,629881394.0
18,b4aaf294-9600-432a-8561-bdebd55d49b3,raw,18,catalog,2019-11-01 22:31:43.019527,629881394.0
19,89e3522e-f8b0-4529-ad5e-dc8ae2021d3b,raw,19,catalog,2019-11-01 22:32:01.596163,629881394.0


In [47]:
len(source_df), len(df3)

(35381, 39132)

In [48]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [49]:
df3.value_counts(['event_type'])

event_type
raw           35381
pause          3751
dtype: int64

In [50]:
print(len(df3[df3['user_id'] == 219483890]))
df3[df3['user_id'] == 219483890].sort_values('event_timestamp')

18


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,0,catalog,2019-11-01 17:59:13.273932,219483890.0
1,072c19d0-427b-4278-809b-c509e52cee4a,raw,1,product1,2019-11-01 17:59:28.459271,219483890.0
2,1692b904-c338-473e-b00b-5b16d5c79d67,raw,2,cart,2019-11-01 17:59:29.502214,219483890.0
3,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,3,catalog,2019-11-01 17:59:32.557029,219483890.0
2407,b2f9c0a1-904a-4dff-90e1-37be04986437,raw,2407,main,2019-12-06 16:22:57.484842,219483890.0
2408,1436f264-7a18-46b6-a732-60201d8b56d8,raw,2408,catalog,2019-12-06 16:23:01.331109,219483890.0
2409,7019613a-00ba-4aa8-b08d-3118638d9c02,raw,2409,catalog,2019-12-06 16:23:48.116617,219483890.0
5169,ebdfbd61-4fcd-44ff-b562-649e3efc76f8,raw,5169,main,2020-01-06 22:10:13.635011,219483890.0
5170,a927160b-96ce-4c89-bd53-d79a37bdd1b6,raw,5170,catalog,2020-01-06 22:10:15.228575,219483890.0
5171,2fa7934a-4e76-4018-994e-1dac03a4464f,raw,5171,cart,2020-01-06 22:10:42.309028,219483890.0


In [51]:
print(len(source_df[source_df['user_id'] == 219483890]))
source_df[source_df['user_id'] == 219483890]

17


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,0,catalog,2019-11-01 17:59:13.273932,219483890
1,072c19d0-427b-4278-809b-c509e52cee4a,raw,1,product1,2019-11-01 17:59:28.459271,219483890
2,1692b904-c338-473e-b00b-5b16d5c79d67,raw,2,cart,2019-11-01 17:59:29.502214,219483890
3,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,3,catalog,2019-11-01 17:59:32.557029,219483890
2245,b2f9c0a1-904a-4dff-90e1-37be04986437,raw,2245,main,2019-12-06 16:22:57.484842,219483890
2246,1436f264-7a18-46b6-a732-60201d8b56d8,raw,2246,catalog,2019-12-06 16:23:01.331109,219483890
2247,7019613a-00ba-4aa8-b08d-3118638d9c02,raw,2247,catalog,2019-12-06 16:23:48.116617,219483890
4836,ebdfbd61-4fcd-44ff-b562-649e3efc76f8,raw,4836,main,2020-01-06 22:10:13.635011,219483890
4837,a927160b-96ce-4c89-bd53-d79a37bdd1b6,raw,4837,catalog,2020-01-06 22:10:15.228575,219483890
4838,2fa7934a-4e76-4018-994e-1dac03a4464f,raw,4838,cart,2020-01-06 22:10:42.309028,219483890


## Lost_pause_cutoff

In [52]:
from src.data_processors_lib.rete import LostPauseEvents, LostPauseParams

lost_cutoff_ = (3000, 's')

lost_pause_events_cutoff = EventsNode(
    LostPauseEvents(params=LostPauseParams(lost_cutoff=lost_cutoff_)))

graph.add_node(
    node=lost_pause_events_cutoff,
    parents=[graph.root]
)

result_cutoff = graph.combine(
    node=lost_pause_events_cutoff
)

result_cutoff.to_dataframe().head(8)

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,0,catalog,2019-11-01 17:59:13.273932,219483890.0
1,072c19d0-427b-4278-809b-c509e52cee4a,raw,1,product1,2019-11-01 17:59:28.459271,219483890.0
2,1692b904-c338-473e-b00b-5b16d5c79d67,raw,2,cart,2019-11-01 17:59:29.502214,219483890.0
3,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,3,catalog,2019-11-01 17:59:32.557029,219483890.0
4,78809682-25ea-4d09-b8fb-008c6da653e1,raw,4,catalog,2019-11-01 21:38:19.283663,964964743.0
5,94c484be-8f3e-4578-b43c-ef62184132ca,raw,5,cart,2019-11-01 21:38:36.761221,964964743.0
6,9930949d-a839-4358-863d-f3e16d77f02b,raw,6,delivery_choice,2019-11-01 21:38:37.564693,964964743.0
7,56c73621-a726-43fc-b1e5-a286c93afd85,raw,7,main,2019-11-01 22:28:54.791683,629881394.0


In [53]:
df4 = result_cutoff.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
print(df4.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())

0
3092


In [54]:
print(len(df4[df4['user_id'] == 964964743]))
df4[df4['user_id'] == 964964743]

14


,event_id,event_type,event_index,event_name,event_timestamp,user_id
4,78809682-25ea-4d09-b8fb-008c6da653e1,raw,4,catalog,2019-11-01 21:38:19.283663,964964743.0
5,94c484be-8f3e-4578-b43c-ef62184132ca,raw,5,cart,2019-11-01 21:38:36.761221,964964743.0
6,9930949d-a839-4358-863d-f3e16d77f02b,raw,6,delivery_choice,2019-11-01 21:38:37.564693,964964743.0
2618,eff948e1-e0e8-4afb-b52f-08936264a027,raw,2618,main,2019-12-09 01:42:22.801831,964964743.0
2619,46834714-9baf-45b4-9be2-0604ae8a93bb,raw,2619,catalog,2019-12-09 01:42:23.617764,964964743.0
2620,323eaaa9-f74c-4b27-8f53-215ac900ac43,raw,2620,product2,2019-12-09 01:42:56.877340,964964743.0
2621,a7137007-a5d6-43af-9461-d28828bee74e,raw,2621,catalog,2019-12-09 01:43:05.436223,964964743.0
2622,44d254c7-3f8f-45a0-9348-42865bc01961,raw,2622,catalog,2019-12-09 01:43:36.923178,964964743.0
2623,dc6ba294-693e-499d-810f-8b37e8704856,raw,2623,product2,2019-12-09 01:43:41.174195,964964743.0
2624,9405e65d-7c43-4473-b1ae-1f38550976f2,raw,2624,cart,2019-12-09 01:43:57.325569,964964743.0


In [55]:
print(len(source_df[source_df['user_id'] == 964964743]))
source_df[source_df['user_id'] == 964964743]

13


,event_id,event_type,event_index,event_name,event_timestamp,user_id
4,78809682-25ea-4d09-b8fb-008c6da653e1,raw,4,catalog,2019-11-01 21:38:19.283663,964964743
5,94c484be-8f3e-4578-b43c-ef62184132ca,raw,5,cart,2019-11-01 21:38:36.761221,964964743
6,9930949d-a839-4358-863d-f3e16d77f02b,raw,6,delivery_choice,2019-11-01 21:38:37.564693,964964743
2439,eff948e1-e0e8-4afb-b52f-08936264a027,raw,2439,main,2019-12-09 01:42:22.801831,964964743
2440,46834714-9baf-45b4-9be2-0604ae8a93bb,raw,2440,catalog,2019-12-09 01:42:23.617764,964964743
2441,323eaaa9-f74c-4b27-8f53-215ac900ac43,raw,2441,product2,2019-12-09 01:42:56.877340,964964743
2442,a7137007-a5d6-43af-9461-d28828bee74e,raw,2442,catalog,2019-12-09 01:43:05.436223,964964743
2443,44d254c7-3f8f-45a0-9348-42865bc01961,raw,2443,catalog,2019-12-09 01:43:36.923178,964964743
2444,dc6ba294-693e-499d-810f-8b37e8704856,raw,2444,product2,2019-12-09 01:43:41.174195,964964743
2445,9405e65d-7c43-4473-b1ae-1f38550976f2,raw,2445,cart,2019-12-09 01:43:57.325569,964964743


In [56]:
print(len(source_df[source_df['user_id'] == 629881394]))
# source_df[source_df['user_id'] == 629881394]
print(len(df4[df4['user_id'] == 629881394]))
df4[df4['user_id'] == 629881394]

48
49


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,56c73621-a726-43fc-b1e5-a286c93afd85,raw,7,main,2019-11-01 22:28:54.791683,629881394.0
9,a445bdd8-3c0d-4b0b-b894-a692f45fbbe0,raw,9,catalog,2019-11-01 22:29:01.049513,629881394.0
11,1170cc7c-64d4-49cf-abeb-32b78a595f19,raw,11,catalog,2019-11-01 22:29:32.322458,629881394.0
13,c1f3b671-ca82-4770-b9b8-3faae2c2e76a,raw,13,catalog,2019-11-01 22:30:09.450839,629881394.0
14,c54f2fcc-2fca-4323-8a54-779fdcdaa11d,raw,14,catalog,2019-11-01 22:31:05.565762,629881394.0
15,3217c680-2e11-47c1-bf45-dabb92c8ab65,raw,15,main,2019-11-01 22:31:08.333560,629881394.0
16,66df2664-a3a3-48fe-abe9-2a2d4e85a62c,raw,16,catalog,2019-11-01 22:31:09.010626,629881394.0
17,b2689f02-542c-4390-8733-7c701afaac0d,raw,17,product1,2019-11-01 22:31:10.416231,629881394.0
18,b4aaf294-9600-432a-8561-bdebd55d49b3,raw,18,catalog,2019-11-01 22:31:43.019527,629881394.0
19,89e3522e-f8b0-4529-ad5e-dc8ae2021d3b,raw,19,catalog,2019-11-01 22:32:01.596163,629881394.0


In [57]:
len(source_df), len(df4)

(35381, 39132)

In [58]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [59]:
df4.value_counts(['event_type'])

event_type
raw           35381
lost           3744
pause             7
dtype: int64

In [60]:
print(len(df4[df4['user_id'] == 219483890]))
df4[df4['user_id'] == 219483890].sort_values('event_timestamp')

18


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,0,catalog,2019-11-01 17:59:13.273932,219483890.0
1,072c19d0-427b-4278-809b-c509e52cee4a,raw,1,product1,2019-11-01 17:59:28.459271,219483890.0
2,1692b904-c338-473e-b00b-5b16d5c79d67,raw,2,cart,2019-11-01 17:59:29.502214,219483890.0
3,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,3,catalog,2019-11-01 17:59:32.557029,219483890.0
2407,b2f9c0a1-904a-4dff-90e1-37be04986437,raw,2407,main,2019-12-06 16:22:57.484842,219483890.0
2408,1436f264-7a18-46b6-a732-60201d8b56d8,raw,2408,catalog,2019-12-06 16:23:01.331109,219483890.0
2409,7019613a-00ba-4aa8-b08d-3118638d9c02,raw,2409,catalog,2019-12-06 16:23:48.116617,219483890.0
5169,ebdfbd61-4fcd-44ff-b562-649e3efc76f8,raw,5169,main,2020-01-06 22:10:13.635011,219483890.0
5170,a927160b-96ce-4c89-bd53-d79a37bdd1b6,raw,5170,catalog,2020-01-06 22:10:15.228575,219483890.0
5171,2fa7934a-4e76-4018-994e-1dac03a4464f,raw,5171,cart,2020-01-06 22:10:42.309028,219483890.0


In [61]:
print(len(source_df[source_df['user_id'] == 219483890]))
source_df[source_df['user_id'] == 219483890]

17


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,0,catalog,2019-11-01 17:59:13.273932,219483890
1,072c19d0-427b-4278-809b-c509e52cee4a,raw,1,product1,2019-11-01 17:59:28.459271,219483890
2,1692b904-c338-473e-b00b-5b16d5c79d67,raw,2,cart,2019-11-01 17:59:29.502214,219483890
3,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,3,catalog,2019-11-01 17:59:32.557029,219483890
2245,b2f9c0a1-904a-4dff-90e1-37be04986437,raw,2245,main,2019-12-06 16:22:57.484842,219483890
2246,1436f264-7a18-46b6-a732-60201d8b56d8,raw,2246,catalog,2019-12-06 16:23:01.331109,219483890
2247,7019613a-00ba-4aa8-b08d-3118638d9c02,raw,2247,catalog,2019-12-06 16:23:48.116617,219483890
4836,ebdfbd61-4fcd-44ff-b562-649e3efc76f8,raw,4836,main,2020-01-06 22:10:13.635011,219483890
4837,a927160b-96ce-4c89-bd53-d79a37bdd1b6,raw,4837,catalog,2020-01-06 22:10:15.228575,219483890
4838,2fa7934a-4e76-4018-994e-1dac03a4464f,raw,4838,cart,2020-01-06 22:10:42.309028,219483890


### Результат

1) [поправила] Нельзя задать единицу измерения отсечки, сейчас тип - lost_cutoff: Optional[List[int]]
2) [не работает] - ломается на этапе импорта из-за параметра функции, если
3) [закомментировала] Общая логика работы такая, что можно проставить lost/pause или по отсечке, или по списку и функции. И то и другое задавать нельзя. Надо допридумать тут логику
    - По отсечке - Если от последнего события в траектории прошло больше времени отсечки, то добавляется событие lost, если меньше - pause
    - lost_users_list - список юзеров, которым проставляется событие lost, остальным - pause, Сейчас по списку проставляется - pause
4) [поправила] Combine - часть событий задваивается, но не очень понятно по какой логике
5) Само событие и timestamp проставляются корректно

## Truncated

In [62]:
from src.data_processors_lib.rete import TruncatedEvents, TruncatedParams
params = {
    'left_truncated_cutoff': (10000, 'm'),
    'right_truncated_cutoff': (10000, 'm')
}

truncated_events = EventsNode(
    TruncatedEvents(params=TruncatedParams(**params)))


graph.add_node(
    node=truncated_events,
    parents=[graph.root]
)

result = graph.combine(
    node=truncated_events
)
result.to_dataframe().head(8)

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,59a69b9b-91e6-4ea6-b338-2d50e9b465f5,raw,0,catalog,2019-11-01 17:59:13.273932,219483890.0
1,072c19d0-427b-4278-809b-c509e52cee4a,raw,1,product1,2019-11-01 17:59:28.459271,219483890.0
2,1692b904-c338-473e-b00b-5b16d5c79d67,raw,2,cart,2019-11-01 17:59:29.502214,219483890.0
3,56a76f25-ed9d-46de-8a4f-235bf534f770,raw,3,catalog,2019-11-01 17:59:32.557029,219483890.0
4,78809682-25ea-4d09-b8fb-008c6da653e1,raw,4,catalog,2019-11-01 21:38:19.283663,964964743.0
5,94c484be-8f3e-4578-b43c-ef62184132ca,raw,5,cart,2019-11-01 21:38:36.761221,964964743.0
6,9930949d-a839-4358-863d-f3e16d77f02b,raw,6,delivery_choice,2019-11-01 21:38:37.564693,964964743.0
7,56c73621-a726-43fc-b1e5-a286c93afd85,raw,7,main,2019-11-01 22:28:54.791683,629881394.0


In [63]:
df5 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df5.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [64]:
df5.value_counts('event_type')

event_type
raw                35381
truncated_right     3688
truncated_left      2883
dtype: int64

In [65]:
print(len(df5[df5['user_id'] == 964964743]))
df5[df5['user_id'] == 964964743]

14


,event_id,event_type,event_index,event_name,event_timestamp,user_id
4,78809682-25ea-4d09-b8fb-008c6da653e1,raw,4,catalog,2019-11-01 21:38:19.283663,964964743.0
5,94c484be-8f3e-4578-b43c-ef62184132ca,raw,5,cart,2019-11-01 21:38:36.761221,964964743.0
6,9930949d-a839-4358-863d-f3e16d77f02b,raw,6,delivery_choice,2019-11-01 21:38:37.564693,964964743.0
2884,eff948e1-e0e8-4afb-b52f-08936264a027,raw,2884,main,2019-12-09 01:42:22.801831,964964743.0
2885,46834714-9baf-45b4-9be2-0604ae8a93bb,raw,2885,catalog,2019-12-09 01:42:23.617764,964964743.0
2886,323eaaa9-f74c-4b27-8f53-215ac900ac43,raw,2886,product2,2019-12-09 01:42:56.877340,964964743.0
2887,a7137007-a5d6-43af-9461-d28828bee74e,raw,2887,catalog,2019-12-09 01:43:05.436223,964964743.0
2888,44d254c7-3f8f-45a0-9348-42865bc01961,raw,2888,catalog,2019-12-09 01:43:36.923178,964964743.0
2889,dc6ba294-693e-499d-810f-8b37e8704856,raw,2889,product2,2019-12-09 01:43:41.174195,964964743.0
2890,9405e65d-7c43-4473-b1ae-1f38550976f2,raw,2890,cart,2019-12-09 01:43:57.325569,964964743.0


In [66]:
print(len(source_df[source_df['user_id'] == 964964743]))
source_df[source_df['user_id'] == 964964743]

13


,event_id,event_type,event_index,event_name,event_timestamp,user_id
4,78809682-25ea-4d09-b8fb-008c6da653e1,raw,4,catalog,2019-11-01 21:38:19.283663,964964743
5,94c484be-8f3e-4578-b43c-ef62184132ca,raw,5,cart,2019-11-01 21:38:36.761221,964964743
6,9930949d-a839-4358-863d-f3e16d77f02b,raw,6,delivery_choice,2019-11-01 21:38:37.564693,964964743
2439,eff948e1-e0e8-4afb-b52f-08936264a027,raw,2439,main,2019-12-09 01:42:22.801831,964964743
2440,46834714-9baf-45b4-9be2-0604ae8a93bb,raw,2440,catalog,2019-12-09 01:42:23.617764,964964743
2441,323eaaa9-f74c-4b27-8f53-215ac900ac43,raw,2441,product2,2019-12-09 01:42:56.877340,964964743
2442,a7137007-a5d6-43af-9461-d28828bee74e,raw,2442,catalog,2019-12-09 01:43:05.436223,964964743
2443,44d254c7-3f8f-45a0-9348-42865bc01961,raw,2443,catalog,2019-12-09 01:43:36.923178,964964743
2444,dc6ba294-693e-499d-810f-8b37e8704856,raw,2444,product2,2019-12-09 01:43:41.174195,964964743
2445,9405e65d-7c43-4473-b1ae-1f38550976f2,raw,2445,cart,2019-12-09 01:43:57.325569,964964743


### Результат

1) [Поправила] Дубликаты - 27880
2) [TODO_analytics] перепутаны местами truncated_left и truncated_right
3) [TODO_analytics] проверить расчет более детально
4) [Поправила] Неправильно проставляются truncated события - для всех юзеров

## PositiveTarget

In [68]:
from src.data_processors_lib.rete import PositiveTarget, PositiveTargetParams
params = {
    'positive_target_events': ['payment_done']
}

positive_targets = EventsNode(
    PositiveTarget(params=PositiveTargetParams(**params)))

graph.add_node(
    node=positive_targets,
    parents=[graph.root]
)

result = graph.combine(
    node=positive_targets
)
result.to_dataframe().head(8)

TypeError: _default_func_positive() takes 2 positional arguments but 3 were given

In [69]:
df6 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df6.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [ ]:
df6['event_type'].value_counts()

In [ ]:
source_df['event_type'].value_counts()

In [ ]:
source_df[source_df['event_name'] == 'payment_done']['user_id'].nunique()

In [ ]:
print(len(df6[df6['user_id'] == 629881394]))
df6[df6['user_id'] == 629881394]

In [ ]:
print(len(source_df[source_df['user_id'] == 629881394]))
source_df[source_df['user_id'] == 629881394]

In [ ]:
source_df[source_df['event_name'] == 'payment_done']

### Результат

1) [Поправила] Дубликаты (Но из-за следующей ошибки еще не тестила)
2) [Не работает] TypeError: _default_func_positive() takes 2 positional arguments but 3 were given
3) [TODO] Нужен кейс по тестированию 2 событий
4) [TODO] Нужен кейс по тестированию пользовательской функции

## NegativeTarget

In [70]:
from src.data_processors_lib.rete import NegativeTarget, NegativeTargetParams
params = {
    'negative_target_events': ['payment_cash']
}

negative_targets = EventsNode(
    NegativeTarget(params=NegativeTargetParams(**params)))

graph.add_node(
    node=negative_targets,
    parents=[graph.root]
)

result = graph.combine(
    node=negative_targets
)


TypeError: _default_func_negative() takes 2 positional arguments but 3 were given

In [71]:
df7 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df6.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [ ]:
df7['event_type'].value_counts()

In [ ]:
source_df['event_type'].value_counts()

In [ ]:
source_df[source_df['event_name'] == 'payment_cash']['user_id'].nunique()

In [ ]:
print(len(df7[df7['user_id'] == 629881394]))
df7[df7['user_id'] == 629881394]

In [ ]:
print(len(source_df[source_df['user_id'] == 629881394]))
source_df[source_df['user_id'] == 629881394]

In [ ]:
source_df[source_df['event_name'] == 'payment_cash']

### Результат

1) [Поправила] Дубликаты (Но из-за следующей ошибки еще не тестила)
2) [Не работает] TypeError: _default_func_negative() takes 2 positional arguments but 3 were given
3) [TODO] Нужен кейс по тестированию 2 событий
4) [TODO] Нужен кейс по тестированию пользовательской функции

## CutPathAfter

In [ ]:
from src.data_processors_lib.rete import CutPathAfterEvent, CutPathAfterEventParams
params = {
    'cutoff_events': ['payment_done'],
    'cut_shift': 0,
    'min_cjm': 0
}

cut_after = EventsNode(
    CutPathAfterEvent(params=CutPathAfterEventParams(**params)))

graph.add_node(
    node=cut_after,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_after
)


In [ ]:
df8 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df8.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

In [ ]:
df8['event_type'].value_counts()

In [ ]:
source_df['event_type'].value_counts()

In [ ]:
source_df[source_df['event_name'] == 'payment_cash']['user_id'].nunique()

In [ ]:
print(len(df8[df8['user_id'] == 629881394]))
df8[df8['user_id'] == 629881394]

In [ ]:
print(len(source_df[source_df['user_id'] == 629881394]))
source_df[source_df['user_id'] == 629881394]

In [ ]:
source_df[source_df['event_name'] == 'payment_cash']

### Результат

1) Ворнинги при отработке функции
2) Не работает

## CutPathBefore

In [ ]:
from src.data_processors_lib.rete import CutPathBeforeEvent, CutPathBeforeEventParams
params = {
    'cutoff_events': ['product2'],
    'cut_shift': 0,
    'min_cjm': 0
}

cut_before = EventsNode(
    CutPathAfterEvent(params=CutPathAfterEventParams(**params)))

graph.add_node(
    node=cut_before,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_before
)


In [ ]:
df9 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df9.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

In [ ]:
df9['event_type'].value_counts()

In [ ]:
source_df['event_type'].value_counts()

In [ ]:
source_df[source_df['event_name'] == 'payment_cash']['user_id'].nunique()

In [ ]:
print(len(df9[df9['user_id'] == 629881394]))
df9[df9['user_id'] == 629881394]

In [ ]:
print(len(source_df[source_df['user_id'] == 629881394]))
source_df[source_df['user_id'] == 629881394]

In [ ]:
source_df[source_df['event_name'] == 'payment_cash']

### Результаты

1) ВОрнинги
2) Не работает

## SplitSessions

In [ ]:
from src.data_processors_lib.rete import SplitSessions, SplitSessionsParams
params = {
    'session_cutoff': (3000, 's'),
    'mark_truncated': True,
    'session_col': 'session_id'
}

split_sessions = EventsNode(
    SplitSessions(params=SplitSessionsParams(**params)))

graph.add_node(
    node=split_sessions,
    parents=[graph.root]
)

result = graph.combine(
    node=split_sessions
)


In [ ]:
df10 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df10.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

In [ ]:
df10['event_type'].value_counts()

In [ ]:
source_df['event_type'].value_counts()

In [ ]:
source_df[source_df['event_name'] == 'payment_cash']['user_id'].nunique()

In [ ]:
print(len(df10[df10['user_id'] == 629881394]))
df10[df10['user_id'] == 629881394]

In [ ]:
print(len(source_df[source_df['user_id'] == 629881394]))
source_df[source_df['user_id'] == 629881394]

In [ ]:
source_df[source_df['event_name'] == 'payment_cash']

### Результат

## Grouping

In [ ]:
from src.data_processors_lib.simple_processors import SimpleGroup

cart_events = EventsNode(
    SimpleGroup({
        "event_name": "add_to_cart",
        "filter": lambda df, schema: df[schema.event_name].isin(["cart"]),
    })
)
graph.add_node(
    node=cart_events,
    parents=[graph.root]
)